<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/Spark/SparkRDD/SparkRDDNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq  > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz > /dev/null 
!pip install -q findspark

In [2]:
!mkdir /content/conf /content/lib
!wget -O /content/conf/log4j.properties https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkRDD/conf/log4j.properties > /dev/null 2>&1
!mv /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf.bk  > /dev/null 2>&1
!wget -O /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkRDD/conf/spark-defaults.conf  > /dev/null 2>&1
!wget -O /content/conf/spark.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkRDD/conf/spark.conf > /dev/null 2>&1

!wget -O /content/lib/logger.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkRDD/lib/logger.py  > /dev/null 2>&1
!wget -O /content/lib/utils.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkRDD/lib/utils.py  > /dev/null 2>&1

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

In [ ]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
#!curl -s http://localhost:4040/api/tunnels

In [17]:
def load_survey_file(sc, url):
   sc.addFile(url)

   linesRDD=sc.textFile('file://'+SparkFiles.get("sample.csv"))

   return linesRDD

In [39]:
from pyspark.sql import *
from pyspark import SparkConf, SparkFiles
from lib.logger import Log4J
from lib.utils import get_spark_app_config
from collections import namedtuple

SurveyRecord = namedtuple("SurveyRecord", ["Age", "Gender", "Country", "State"])

conf=get_spark_app_config()

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

sc = spark.sparkContext

logger = Log4J(spark)

logger.info("Staring Application")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkRDD/data/sample.csv'

linesRDD=load_survey_file(sc, url)

partitionedRDD=linesRDD.repartition(2)

colsRDD=partitioned_RDD.map(lambda line: line.replace('"', '').split(","))
selectRDD=colsRDD.map(lambda cols: SurveyRecord(int(cols[1]), cols[2], cols[3], cols[4]))
filteredRDD=selectRDD.filter(lambda r: r.Age < 40)
kvRDD=filteredRDD.map(lambda r: (r.Country, 1))
countRDD=kvRDD.reduceByKey(lambda v1, v2: v1+v2)

logger.info(countRDD.collect())

logger.info("Ending Application")

In [ ]:
!cat app-logs/sparklog.log